**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



**Set global variables**

In [2]:
FP_REGION_LABEL_A = file.path(
    FD_RES, 
    "region/summary/metadata.label.astarr_macs.tsv"
)
FP_REGION_LABEL_B = file.path(
    FD_RES,
    "region/genome_tss/summary/metadata.label.tsv"
)

## Import data

In [3]:
dat_region_label_A = read_tsv(FP_REGION_LABEL_A, show_col_types = FALSE)
dat_region_label_A

Folder,FName,Label
<chr>,<chr>,<chr>
fcc_astarr_macs,K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.bed.gz,fcc_astarr_macs_input_overlap
fcc_astarr_macs,K562.hg38.ASTARR.macs.KS91.input.rep_all.union.q5.bed.gz,fcc_astarr_macs_input_union


In [4]:
dat_region_label_B = read_tsv(FP_REGION_LABEL_B, show_col_types = FALSE)
dat_region_label_B

Folder,FName,Label
<chr>,<chr>,<chr>
genome_tss,K562.hg38.TSS.selected_by_highest_Pol2_signal.bed.gz,genome_tss_pol2
genome_tss,K562.hg38.TSS.selected_by_highest_Pol2_signal.filtered_by_RNAseq_TPM.bed.gz,genome_tss_pol2_rnaseq


## Main

**init**

In [5]:
dat = dat_region_label_A
lst = split(dat, 1:nrow(dat))
lst = lapply(lst, as.list)
lst_region_label_A = lst
lst

$`1`
$`1`$Folder
[1] "fcc_astarr_macs"

$`1`$FName
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.bed.gz"

$`1`$Label
[1] "fcc_astarr_macs_input_overlap"


$`2`
$`2`$Folder
[1] "fcc_astarr_macs"

$`2`$FName
[1] "K562.hg38.ASTARR.macs.KS91.input.rep_all.union.q5.bed.gz"

$`2`$Label
[1] "fcc_astarr_macs_input_union"

In [7]:
dat = dat_region_label_B
vec = unique(dat$Folder)
vec_txt_region_folder_B = vec
for(txt in vec){cat(txt, "\n")}

genome_tss 


**Loop**

In [16]:
### loop through region A
for (idx in seq_along(lst_region_label_A)){
    
    ### get region A
    lst = lst_region_label_A[[idx]]
    txt_region_folder_A = lst$Folder
    txt_region_label_A  = lst$Label

    ### get column names for region A
    txt_fdiry = file.path(FD_RES, "region", txt_region_folder_A, "summary")
    txt_fname = "description.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)
    dat_cname = read_tsv(txt_fpath, show_col_types = FALSE)
    vec_txt_cname_A = dat_cname$Name

    ### loop through region B
    for (txt_region_folder_B in vec_txt_region_folder_B){
        
        ### get column names for region B
        txt_fdiry = file.path(FD_RES, "region", txt_region_folder_B, "summary")
        txt_fname = "description.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat_cname = read_tsv(txt_fpath, show_col_types = FALSE)
        vec_txt_cname_B = dat_cname$Name

        ### get total column names for annotation table
        vec_txt_cname = c(
            paste0(vec_txt_cname_A, "_A"),
            paste0(vec_txt_cname_B, "_B"),
            "Distance"
        )

        ### set file directory
        txt_fdiry = file.path(
            FD_RES, 
            "region_closest",
            txt_region_label_A, 
            txt_region_folder_B
        )
        txt_fname = "*bed.gz"
        txt_fglob = file.path(txt_fdiry, txt_fname)

        ### get region pair table files
        vec_txt_fpath = Sys.glob(txt_fglob)
        vec_txt_fname = basename(vec_txt_fpath)

        ### show progress
        cat("===========================================\n")
        cat("Read region pairs...", "\n")
        cat("Region A:", txt_region_folder_A, "|", txt_region_label_A, "\n")
        cat("Region B:", txt_region_folder_B, "\n")
        cat("FDiry:   ", "\n")
        print(txt_fdiry)
        cat("FName:   ", "\n")
        print(vec_txt_fname)
        cat("\n")
        flush.console()

        ### import region pair tables
        lst = lapply(vec_txt_fname, function(txt_fname){
            ### set file directory
            txt_fpath = file.path(txt_fdiry, txt_fname)
            
            ### get annotation region A and region B
            vec = str_split(txt_fname, "\\.")[[1]]
            txt_annot_A = vec[1]
            txt_annot_B = vec[2]

            ### read table and add annotation labels
            dat = read_tsv(txt_fpath, col_names = vec_txt_cname, show_col_types = FALSE)
            dat = dat %>% 
                dplyr::mutate(
                    Region_A = fun_gen_region(Chrom_A, ChromStart_A, ChromEnd_A),
                    Region_B = fun_gen_region(Chrom_B, ChromStart_B, ChromEnd_B),
                    Annotation_A = txt_annot_A,
                    Annotation_B = txt_annot_B
                )
            return(dat)
        }) 

        ### concatenate tables and assign
        dat = bind_rows(lst)
        dat_region_annot_pair = dat
        
        ### set directory
        txt_fdiry = file.path(
            FD_RES, 
            "region_closest",
            txt_region_label_A,
            "summary"
        )
        txt_fname = paste("region", "pair",  txt_region_folder_B, "tsv", sep = ".")
        txt_fpath = file.path(txt_fdiry, txt_fname)
        
        ### show progress
        cat("Save region pairs...",  "\n")
        cat("FDiry:", txt_fdiry, "\n")
        cat("FName:", txt_fname, "\n")
        cat("\n")
        flush.console()

        ### write tables
        dir.create(txt_fdiry, showWarnings = FALSE)
        write_tsv(dat_region_annot_pair, txt_fpath)
    }
}

Read region pairs... 
Region A: fcc_astarr_macs | fcc_astarr_macs_input_overlap 
Region B: genome_tss 
FDiry:    
[1] "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_closest/fcc_astarr_macs_input_overlap/genome_tss"
FName:    
[1] "fcc_astarr_macs_input_overlap.genome_tss_pol2.bed.gz"       
[2] "fcc_astarr_macs_input_overlap.genome_tss_pol2_rnaseq.bed.gz"

Save region pairs... 
FDiry: /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_closest/fcc_astarr_macs_input_overlap/summary 
FName: region.pair.genome_tss.tsv 

Read region pairs... 
Region A: fcc_astarr_macs | fcc_astarr_macs_input_union 
Region B: genome_tss 
FDiry:    
[1] "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_closest/fcc_astarr_macs_input_union/genome_tss"
FName:    
[1] "fcc_astarr_macs_input_union.genome_tss_pol2.bed.gz"       
[2] "fcc_astarr_macs_input_union.genome_tss_pol2_rnaseq.bed.gz"

Save region pairs... 
FDiry: /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_closest/fcc_astar

In [4]:
txt_fdiry = TXT_FDIRY_REGION_LABEL
txt_fname = TXT_FNAME_REGION_LABEL_A
txt_fpath = file.path(txt_fdiry, txt_fname)

dat_region_label_A = read_tsv(txt_fpath, show_col_types = FALSE)
dat_region_label_A

Folder,FName,Label
<chr>,<chr>,<chr>
fcc_astarr_macs,K562.hg38.ASTARR.macs.KS91.input.rep_all.max_overlaps.q5.bed.gz,fcc_astarr_macs_input_overlap
fcc_astarr_macs,K562.hg38.ASTARR.macs.KS91.input.rep_all.union.q5.bed.gz,fcc_astarr_macs_input_union


In [5]:
txt_fdiry = TXT_FDIRY_REGION_LABEL
txt_fname = TXT_FNAME_REGION_LABEL_B
txt_fpath = file.path(txt_fdiry, txt_fname)

dat_region_label_B = read_tsv(txt_fpath, show_col_types = FALSE)
dat_region_label_B

Folder,FName,Label
<chr>,<chr>,<chr>
encode_chromatin_states,K562.hg38.cCREs.silencer_rest.bed.gz,encode_ccres_silencer_rest
encode_chromatin_states,K562.hg38.cCREs.silencer_starr.bed.gz,encode_ccres_silencer_starr
encode_chromatin_states,K562.hg38.ENCSR365YNI.ENCFF106BGJ.ChromHMM.simplified.bed.gz,encode_chromhmm_ENCFF106BGJ
encode_chromatin_states,K562.hg38.ENCSR913HQX.ENCFF286VQG.cCREs.simplified.bed.gz,encode_ccres_ENCFF286VQG
encode_e2g_benchmark,K562.hg38.ENCODE_E2G.benchmark.bed.gz,encode_e2g_benchmark
fcc_astarr_csaw,K562.hg38.ASTARR.csaw.KS91.bed.gz,fcc_astarr_csaw_KS91
fcc_astarr_csaw,K562.hg38.ASTARR.csaw.KSMerge.bed.gz,fcc_astarr_csaw_KSMerge
fcc_crispri_growth,K562.hg38.CRISPRi_Growth.signif.bed.gz,fcc_crispri_growth_signif
fcc_crispri_growth,K562.hg38.CRISPRi_Growth.total.bed.gz,fcc_crispri_growth_total
